# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [26]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np

In [27]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [28]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [29]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_to = pd.read_csv(file_path)

In [30]:
to_data.head()

,neighbourhood,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented
year,,,,,,,,,,,,
2001,Agincourt North,3715,1480,0,1055,1295,195,185,5,200388,810,870
2001,Agincourt South-Malvern West,3250,1835,0,545,455,105,425,0,203047,806,892
2001,Alderwood,3175,315,0,470,50,185,370,0,259998,817,924
2001,Annex,1060,6090,5,1980,605,275,3710,165,453850,1027,1378
2001,Banbury-Don Mills,3615,4465,0,240,380,15,1360,0,371864,1007,1163


In [31]:
df_to.head()

,neighbourhood,lat,lon
0,Agincourt North,43.805441,-79.266712
1,Agincourt South-Malvern West,43.788658,-79.265612
2,Alderwood,43.604937,-79.541611
3,Annex,43.671585,-79.404001
4,Banbury-Don Mills,43.737657,-79.349718


## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [32]:
# Getting the data from the top 10 expensive neighbourhoods
expensive_house = to_data[["neighbourhood",'single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house',"average_house_value","shelter_costs_owned","shelter_costs_rented"]].groupby("neighbourhood").mean().sort_values(by='average_house_value',ascending=False)
expensive_house = expensive_house[:10]
expensive_house

,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented
neighbourhood,,,,,,,,,,,
Bridle Path-Sunnybrook-York Mills,2260.00,331.25,0.00,36.25,90.00,25.0,40.00,0.00,1526485.75,2360.75,2321.75
Forest Hill South,1742.50,2031.25,1.25,61.25,45.00,75.0,1027.50,3.75,1195992.50,1781.00,1313.75
Lawrence Park South,3472.50,773.75,0.00,126.25,38.75,225.0,966.25,16.25,1094027.75,1954.00,1372.75
Rosedale-Moore Park,2498.75,4641.25,0.00,486.25,245.00,327.5,1618.75,2.50,1093640.00,1909.75,1537.25
St.Andrew-Windfields,3225.00,1670.00,0.00,185.00,552.50,97.5,586.25,5.00,999107.00,1880.25,1384.50
Casa Loma,916.25,2310.00,0.00,288.75,201.25,162.5,1192.50,2.50,981064.25,1873.75,1547.75
Bedford Park-Nortown,4865.00,1981.25,0.00,43.75,57.50,287.5,1275.00,88.75,930415.25,1786.75,1255.00
Forest Hill North,1488.75,3392.50,0.00,12.50,16.25,82.5,402.50,1.25,851680.50,1722.75,1245.50
Kingsway South,2326.25,576.25,0.00,66.25,48.75,20.0,336.25,2.50,843234.25,1736.75,1622.00


In [33]:
# Calculate the mean number of dwelling types units per year
dwelling_types_year = to_data[["neighbourhood",'single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house']].groupby([to_data.index,"neighbourhood"]).sum().reset_index()
dwelling_types_year.set_index("year", inplace=True)
dwelling_types_year[:5]

,neighbourhood,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house
year,,,,,,,,,
2001,Agincourt North,3715,1480,0,1055,1295,195,185,5
2001,Agincourt South-Malvern West,3250,1835,0,545,455,105,425,0
2001,Alderwood,3175,315,0,470,50,185,370,0
2001,Annex,1060,6090,5,1980,605,275,3710,165
2001,Banbury-Don Mills,3615,4465,0,240,380,15,1360,0


In [34]:
# Calculate the average monthly shelter costs for owned and rented dwellings
num_dwelling_type = to_data[['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house']].groupby(to_data.index).sum()
num_dwelling_type

,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house
year,,,,,,,,
2001,300930,355015,75,90995,52355,23785,116900,3040
2006,266860,379400,165,69430,54690,44095,162850,1335
2011,274940,429220,100,72480,60355,44750,163895,2165
2016,269680,493270,95,71200,61565,48585,165575,2845


In [35]:
year_2001 = pd.DataFrame(num_dwelling_type.iloc[0,:])
year_2001.rename(columns={2001:'num_dwelling'}, inplace=True)

year_2006 = pd.DataFrame(num_dwelling_type.iloc[1,:])
year_2006.rename(columns={2006:'num_dwelling'}, inplace=True)

year_2011 = pd.DataFrame(num_dwelling_type.iloc[2,:])
year_2011.rename(columns={2011:'num_dwelling'}, inplace=True)

year_2016 = pd.DataFrame(num_dwelling_type.iloc[3,:])
year_2016.rename(columns={2016:'num_dwelling'}, inplace=True)

In [36]:
avg_house_value = to_data[['average_house_value']].groupby(to_data.index).mean()
avg_house_value

,average_house_value
year,
2001,289882.885714
2006,424059.664286
2011,530424.721429
2016,664068.328571


In [37]:
shelter_avg_costs = to_data[['shelter_costs_owned', 'shelter_costs_rented']].groupby(to_data.index).mean()

shelter_avg_costs

,shelter_costs_owned,shelter_costs_rented
year,,
2001,846.878571,1085.935714
2006,1316.800000,925.414286
2011,1448.214286,1019.792857
2016,1761.314286,1256.321429


In [38]:
# Combined data frame
neighbourhood =  to_data[['neighbourhood','single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house',"average_house_value","shelter_costs_owned","shelter_costs_rented"]].groupby("neighbourhood").mean().reset_index()
combined_df = pd.concat([df_to.set_index('neighbourhood'),neighbourhood.set_index('neighbourhood')], axis=1, join='inner').reset_index()
combined_df.head()

,neighbourhood,lat,lon,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented
0,Agincourt North,43.805441,-79.266712,3435.00,1947.50,2.50,863.75,1406.25,512.50,547.50,10.00,329811.5,1109.00,983.50
1,Agincourt South-Malvern West,43.788658,-79.265612,2897.50,2180.00,1.25,375.00,456.25,523.75,628.75,32.50,334189.0,1131.25,985.00
2,Alderwood,43.604937,-79.541611,2903.75,302.50,1.25,503.75,76.25,302.50,502.50,1.25,427922.5,1166.75,1003.25
3,Annex,43.671585,-79.404001,751.25,7235.00,1.25,1375.00,613.75,355.00,4605.00,83.75,746977.0,1692.75,1315.25
4,Banbury-Don Mills,43.737657,-79.349718,3572.50,5388.75,1.25,273.75,626.25,32.50,1340.00,0.00,612039.0,1463.50,1242.75


In [39]:
by_neighbourhood = to_data[['neighbourhood','average_house_value']].groupby([to_data.index,"neighbourhood"]).mean().reset_index()
by_neighbourhood.set_index('year', inplace=True)
by_neighbourhood[:10]

,neighbourhood,average_house_value
year,,
2001,Agincourt North,200388
2001,Agincourt South-Malvern West,203047
2001,Alderwood,259998
2001,Annex,453850
2001,Banbury-Don Mills,371864
2001,Bathurst Manor,304749
2001,Bay Street Corridor,257404
2001,Bayview Village,327644
2001,Bayview Woods-Steeles,343535


In [40]:
costs_per_neighbourhood = to_data.filter(["neighbourhood","shelter_costs_owned","shelter_costs_rented"], axis =1)
costs_per_neighbourhood

,neighbourhood,shelter_costs_owned,shelter_costs_rented
year,,,
2001,Agincourt North,810,870
2001,Agincourt South-Malvern West,806,892
2001,Alderwood,817,924
2001,Annex,1027,1378
2001,Banbury-Don Mills,1007,1163
2001,Bathurst Manor,843,1052
2001,Bay Street Corridor,1218,1142
2001,Bayview Village,1197,1164
2001,Bayview Woods-Steeles,1212,1018


In [41]:
top_neighbourhood = expensive_house.reset_index()
top_neighbourhood = top_neighbourhood["neighbourhood"].tolist()
top_neighbourhood

['Bridle Path-Sunnybrook-York Mills',
 'Forest Hill South',
 'Lawrence Park South',
 'Rosedale-Moore Park',
 'St.Andrew-Windfields',
 'Casa Loma',
 'Bedford Park-Nortown',
 'Forest Hill North',
 'Kingsway South',
 'Yonge-St.Clair']

In [42]:
top_neighbourhood_df = costs_per_neighbourhood.query("neighbourhood in @top_neighbourhood")
top_neighbourhood_df

,neighbourhood,shelter_costs_owned,shelter_costs_rented
year,,,
2001,Bedford Park-Nortown,933,1491
2001,Bridle Path-Sunnybrook-York Mills,1983,1790
2001,Casa Loma,1241,1500
2001,Forest Hill North,940,1428
2001,Forest Hill South,1001,1469
2001,Kingsway South,1362,1340
2001,Lawrence Park South,1021,1630
2001,Rosedale-Moore Park,1219,1540
2001,St.Andrew-Windfields,1055,1551


In [43]:
sb_df = top_neighbourhood_df.reset_index()
sb_df

,year,neighbourhood,shelter_costs_owned,shelter_costs_rented
0,2001,Bedford Park-Nortown,933,1491
1,2001,Bridle Path-Sunnybrook-York Mills,1983,1790
2,2001,Casa Loma,1241,1500
3,2001,Forest Hill North,940,1428
4,2001,Forest Hill South,1001,1469
5,2001,Kingsway South,1362,1340
6,2001,Lawrence Park South,1021,1630
7,2001,Rosedale-Moore Park,1219,1540
8,2001,St.Andrew-Windfields,1055,1551
9,2001,Yonge-St.Clair,1091,1351


In [44]:
year = sb_df['year']
neighbourhood = sb_df['neighbourhood']
shelter_costs_owned = sb_df['shelter_costs_owned']
shelter_costs_rented = sb_df['shelter_costs_rented']

In [52]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
    map_1 = px.scatter_mapbox(
    combined_df,
    lat="lat",
    lon="lon",
    color="average_house_value",
    size="average_house_value",
    color_continuous_scale=px.colors.sequential.Cividis
    )
    return map_1

def create_bar_2001():
    bar_plot = year_2001.hvplot.bar(xlabel='2001',ylabel='Dwelling Types in Toronto in 2001',title='Dwelling Types in Toronto in 2001', width=650, height=500, color='red',rot=90).opts(yformatter='%.0f2')
    return bar_plot

def create_bar_2006():
    bar_plot = year_2006.hvplot.bar(xlabel='2001',ylabel='Dwelling Types in Toronto in 2006', title='Dwelling Types in Toronto in 2006',width=650, height=500, color='blue',rot=90).opts(yformatter='%.0f2')
    return bar_plot

def create_bar_2011():
    bar_plot = year_2011.hvplot.bar(xlabel='2001',ylabel='Dwelling Types in Toronto in 2011', title='Dwelling Types in Toronto in 2011',width=650, height=500, color='orange',rot=90).opts(yformatter='%.0f2')
    return bar_plot

def create_bar_2016():
    bar_plot = year_2016.hvplot.bar(xlabel='2001',ylabel='Dwelling Types in Toronto in 2016', title='Dwelling Types in Toronto in 2016',width=650, height=500, color='purple',rot=90).opts(yformatter='%.0f2')
    return bar_plot

def create_line_chart_owned():
    line_plot = shelter_avg_costs['shelter_costs_owned'].hvplot.line(xlabel="Year", ylabel="Avg Monthly Shelter Costs", title='Average Monthly Shelter Costs for owned Dwellling in Toronto',color="blue")
    return line_plot

def create_line_chart_rented():
    line_plot = shelter_avg_costs['shelter_costs_rented'].hvplot.line(xlabel="Year", ylabel="Avg Monthly Shelter Costs",title='Average Monthly Shelter Costs for rented Dwellling in Toronto', color="orange")
    return line_plot

def average_house_value():
    avg_house_plot = avg_house_value['average_house_value'].hvplot.line(xlabel="Year", ylabel="Avg Monthly Shelter Costs",title='Average House Value in Toronto', color="Green")
    return avg_house_plot

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    by_neighbourhood_plot = by_neighbourhood.hvplot(groupby='neighbourhood', xlabel='Year', ylabel='Avg House Value').opts(yformatter="%.0f")
    return by_neighbourhood_plot
    
def number_dwelling_types():
    """Number of dwelling types per year"""
    dwell_type_plot = dwelling_types_year.hvplot.bar(groupby="neighbourhood", rot=90,width=650, height=500,xlabel='Year', ylabel='Dwelling Types Units')
    return dwell_type_plot

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    by_neighbourhood_hv = px.bar(by_neighbourhood,x='neighbourhood',y='average_house_value',facet_row=by_neighbourhood.index,title='Average House Values in Toronto per Neighbourhood',height=1200, width=900, hover_data=['average_house_value'], color='average_house_value')
    return by_neighbourhood_hv

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    top_10_plot = expensive_house.hvplot.bar(x='neighbourhood', y='average_house_value', rot=90, height=500,title='Top10 Expensive Neighbourhoods in Toronto', xlabel='Neighbourhood', ylabel='Avg House Value')
    return top_10_plot

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    fig = px.sunburst(sb_df, path=[year, neighbourhood],values=shelter_costs_owned,color=shelter_costs_owned,hover_data=[shelter_costs_rented], color_continuous_scale=['red', 'yellow', 'green'],title='Costs Analysis of Most Expensive Neighbourhood in Toronto per Year')
    return fig

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [56]:
dashboard_title ="# Real Estate Analysis for Toronto: 2001 to 2016"

dashboard_welcome ="The dashboard presents a visual analysis of historical house values, dwelling types per neighbourhood and dwelling costs in Toronto, Ontario from 2001 to 2016"
dashboard_page = pn.Column(dashboard_title, dashboard_welcome, neighbourhood_map())
yearly_analysis = pn.Column(create_bar_2001(), create_bar_2006(), create_bar_2011(),create_bar_2016())
shelter_home_value = pn.Column(create_line_chart_owned(),create_line_chart_rented(), average_house_value())
house_value_dwelling = pn.Column(average_value_by_neighbourhood(), number_dwelling_types(),average_house_value_snapshot())
welcome_plot = pn.Column(dashboard_title,dashboard_welcome, neighbourhood_map())
expensive_neighbourhood = pn.Column(top_most_expensive_neighbourhoods(), sunburts_cost_analysis())

dashboard_tabs = pn.Tabs(             
                ("Home", dashboard_page),
                ("Yearly Market Analysis", yearly_analysis),
                ("Shelter Costs vs Home Value", shelter_home_value),
                ("Neighbourhood Analysis", house_value_dwelling),
                ("Top Expensive Neighbourhoods", expensive_neighbourhood)
)

AttributeError: type object 'object' has no attribute 'dtype'

## Serve the Panel Dashboard

In [55]:
dashboard_tabs.servable()

NameError: name 'dashboard_tabs' is not defined

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()